In [1]:
# Install required packages
!pip install google-generativeai tavily-python

import os
import time
from typing import List
from dataclasses import dataclass
import google.generativeai as genai
from tavily import TavilyClient
import getpass

# Data Classes
@dataclass
class SearchResult:
    title: str
    content: str
    url: str

@dataclass
class ResearchQuestion:
    question: str
    search_results: List[SearchResult]
    summary: str = ""

# ReAct Agent Class
class ReActAgent:
    def __init__(self, gemini_api_key, tavily_api_key):
        self.research_questions: List[ResearchQuestion] = []
        genai.configure(api_key=gemini_api_key)
        self.llm_client = genai.GenerativeModel("gemini-1.5-flash")
        self.search_client = TavilyClient(api_key=tavily_api_key)
        print("✅ ReAct Agent initialized successfully")

    def _call_gemini(self, prompt: str) -> str:
        try:
            response = self.llm_client.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7,
                    max_output_tokens=1000,
                )
            )
            return response.text.strip() if response.text else ""
        except Exception:
            try:
                alt_client = genai.GenerativeModel("gemini-3.5-pro")
                response = alt_client.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0.7,
                        max_output_tokens=1000,
                    )
                )
                return response.text.strip() if response.text else ""
            except Exception:
                return ""

    def generate_research_questions(self, topic: str, num_questions: int = 5) -> List[str]:
        prompt = f"""Generate {num_questions} specific research questions about "{topic}".
Each question should be clear, cover different aspects, and be formatted as a numbered list."""
        response = self._call_gemini(prompt)
        questions = [line.split('.', 1)[-1].strip() for line in response.split('\n') if line.strip() and line[0].isdigit()]
        return questions[:num_questions]

    def search_web(self, query: str) -> List[SearchResult]:
        try:
            search_response = self.search_client.search(query=query, max_results=3)
            return [
                SearchResult(
                    title=result.get('title', ''),
                    content=result.get('content', ''),
                    url=result.get('url', '')
                )
                for result in search_response.get('results', [])
            ]
        except Exception:
            return []

    def summarize_findings(self, question: str, search_results: List[SearchResult]) -> str:
        if not search_results:
            return "No information found."
        context = "\n\n".join(
            f"Source {i+1}: {res.title}\n{res.content[:500]}..."
            for i, res in enumerate(search_results)
        )
        prompt = f"""Answer this question based on the search results provided:

Question: {question}

Search Results:
{context}

Instructions:
- Provide a 2-3 paragraph answer using the sources.
- Be factual and objective.
- If sources conflict, mention differing perspectives.

Answer:"""
        return self._call_gemini(prompt) or "Unable to generate summary."

    def research_topic(self, topic: str, num_questions: int = 5):
        print(f"🚀 Researching: {topic}\n" + "=" * 50)
        questions = self.generate_research_questions(topic, num_questions)
        if not questions:
            print("❌ Failed to generate questions. Check API key or connection.")
            return

        for i, question in enumerate(questions, 1):
            print(f"\n📋 Question {i}: {question}")
            search_results = self.search_web(question)
            summary = self.summarize_findings(question, search_results)
            self.research_questions.append(ResearchQuestion(question, search_results, summary))
            print(f"📝 Summary: {summary[:100]}...")
            time.sleep(1)

        print("\n✅ Research completed!")

    def generate_report(self, topic: str) -> str:
        if not self.research_questions:
            return "No research data available."

        report = f"# Research Report: {topic}\n\n"
        report += "## Introduction\n"
        report += f"This report presents findings on \"{topic}\" using the ReAct (Reasoning + Acting) approach.\n\n"
        report += "## Research Questions and Findings\n"

        for i, rq in enumerate(self.research_questions, 1):
            report += f"\n### {i}. {rq.question}\n\n{rq.summary}\n\n"
            if rq.search_results:
                report += "**Sources:**\n"
                report += "\n".join(f"{j+1}. [{r.title}]({r.url})" for j, r in enumerate(rq.search_results))
                report += "\n\n"

        report += "## Conclusion\n"
        report += f"This research explored {len(self.research_questions)} key aspects of {topic}.\n"
        return report

# Main Execution
print("Enter your API keys:")
GEMINI_API_KEY = getpass.getpass("Gemini API Key: ")
TAVILY_API_KEY = getpass.getpass("Tavily API Key: ")

try:
    agent = ReActAgent(GEMINI_API_KEY, TAVILY_API_KEY)
except Exception as e:
    print(f"Error initializing agent: {e}")
    exit()

topic = input("🔎 Enter the topic you want to research: ")

try:
    agent.research_topic(topic, num_questions=5)
except Exception as e:
    print(f"Error during research: {e}")
    exit()

report = agent.generate_report(topic)
print("\n" + "="*60)
print("FINAL REPORT")
print("="*60)
print(report)

filename = f"research_report_{topic.replace(' ', '_')}.md"
with open(filename, 'w') as f:
    f.write(report)

print(f"\n📄 Report saved as: {filename}")


Enter your API keys:
Gemini API Key: ··········
Tavily API Key: ··········
✅ ReAct Agent initialized successfully
🔎 Enter the topic you want to research: SNS Square 
🚀 Researching: SNS Square 

📋 Question 1: **What is the correlation between the frequency of SNS Square usage and the reported levels of social isolation among its users, controlling for demographic variables such as age, gender, and location?** (This question investigates the potential paradoxical effect of social media on social isolation.)
📝 Summary: The provided text offers limited direct information regarding the correlation between SNS Square usa...

📋 Question 2: **How does the algorithmic curation of content on SNS Square influence the formation and reinforcement of echo chambers and filter bubbles, and what are the observable consequences on users' political attitudes and beliefs?** (This question explores the impact of algorithms on information consumption and polarization.)
📝 Summary: SNS Square's algorithmic co